In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class exact_binomial_tree:

    def __init__(self, df, nfuture, last_price , npath, rfr, risk_neutral = True):
        self.nfuture = nfuture
        self.df = df
        self.rfr = rfr
        self.risk_neutral = risk_neutral
        self.npath = nfuture
        self.last_price = last_price

        self.u = None
        self.d = None
        self.p = None

    def plot_simulated_price_paths(self, simulated_prices):
        # Plotting
        fig, ax = plt.subplots(figsize=(10, 6))

        # Plot price nodes and option values
        for i in range(self.nfuture + 1):
            for j in range(i + 1):
                S = simulated_prices[j, i]
                ax.plot(i, S, 'k', linewidth=1, alpha=0.6)  # Stock price node

                # Draw lines between nodes
                if i < self.nfuture:
                    ax.plot([i, i + 1], [S, simulated_prices[j, i + 1]], 'k--', linewidth=1, alpha=0.6)       # down move
                    ax.plot([i, i + 1], [S, simulated_prices[j + 1, i + 1]], 'k--', linewidth=1, alpha=0.6)   # up move

        ax.set_title(f"Simulated Future Price Paths ({self.nfuture} Days Ahead)",
          fontsize=16, fontweight='bold')
        ax.set_xlabel("Time Step")
        ax.set_ylabel("Stock Price")
        ax.grid(True)
        plt.tight_layout()
        plt.show()

    def simulate_future_prices_CRR(self, mu_sample, sigma_sample):
        # Initialize a 2D array for all simulations (each row is one simulation)
        simulated_prices = np.zeros((self.nfuture + 1, self.nfuture + 1))

        dt = 1 / 252  # Daily time step

        u = np.exp(sigma_sample * np.sqrt(dt))
        d = 1 / u

        if self.risk_neutral:
            p = (np.exp(self.rfr * dt) - d) / (u - d)
        else:
            p = (np.exp(mu_sample * dt) - d) / (u - d)

        for i in range(0, self.nfuture + 1):
            for j in range(i+1):
                # Update the simulated price path
                simulated_prices[j, i] = self.last_price * (u**j) * (d**(i-j))

        self.u = u
        self.d = d
        self.p = p
        return simulated_prices

    def price_vanilla_options(self, simulated_prices, strike, option_type, exercise_type, exercise_days):
        S_maturity = simulated_prices[:, -1]
        maturity_years = self.nfuture / 252
        dt = 1 / 252  # Daily time step

        # Get payoffs at maturity
        if option_type == 'call':
            payoff = np.maximum(S_maturity - strike, 0)
        else:
            payoff = np.maximum(strike - S_maturity, 0)

        option_price = np.zeros_like(simulated_prices)
        option_price[:, -1] = payoff

        if exercise_type == 'european':
            exercise_binary = np.zeros(self.nfuture + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(self.nfuture + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(self.nfuture + 1)
            for i in range(self.nfuture+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Work backward through the tree
        for i in range(self.nfuture - 1, -1, -1):

            #expectation of discounted option prices in the future
            continuation = np.exp(-self.rfr * dt) * (self.p * payoff[1:] + (1 - self.p) * payoff[:-1])

            # Recalculate stock prices at this step
            S_maturity = S_maturity[:i+1] / self.u  # Going one step backward in the tree

            if exercise_binary[i] == 1:
                # Apply early exercise condition
                if option_type == 'call':
                    payoff = np.maximum(continuation, S_maturity - strike)
                    option_price [:len(payoff), i] = payoff
                else:
                    payoff = np.maximum(continuation, strike - S_maturity)
                    option_price [:len(payoff), i] = payoff
            else:
                payoff = continuation
                option_price [:len(continuation), i] = payoff

        # Final result at the root of the tree
        cal_price = payoff[0]
        return cal_price, option_price

    def compute_binomial_arithmetic_averages(self, simulated_prices):
        n = simulated_prices.shape[1] - 1
        arith_avg = np.zeros_like(simulated_prices)

        for j in range(n + 1):
            i_vals = np.arange(j + 1)
            path_prices = np.zeros((j + 1, j + 1))

            for k in range(j + 1):
                up_at_k = np.maximum(0, i_vals - (j - k))
                path_prices[:, k] = simulated_prices[up_at_k, k]

            arith_avg[:j + 1, j] = np.mean(path_prices, axis=1)

        return arith_avg


    def compute_binomial_geometric_averages(self, simulated_prices):
        n = simulated_prices.shape[1] - 1
        geo_avg = np.zeros_like(simulated_prices)

        for j in range(n + 1):  # time step
            i_vals = np.arange(j + 1)
            log_prices = np.zeros((j + 1, j + 1))

            for k in range(j + 1):
                up_at_k = np.maximum(0, i_vals - (j - k))
                log_prices[:, k] = np.log(simulated_prices[up_at_k, k])

            geo_avg[:j + 1, j] = np.exp(np.mean(log_prices, axis=1))

        return geo_avg

    def price_asian_options(self, simulated_prices, strike, option_type, exercise_type, exercise_days):

        Saverages = self.compute_binomial_geometric_averages(simulated_prices)
        #Saverages = self.compute_binomial_arithmetic_averages(simulated_prices)

        S_maturity = Saverages[:, -1]
        maturity_years = self.nfuture / 252
        dt = 1 / 252  # Daily time step

        # Get payoffs at maturity
        if option_type == 'call':
            payoff = np.maximum(S_maturity - strike, 0)
        else:
            payoff = np.maximum(strike - S_maturity, 0)

        option_price = np.zeros_like(simulated_prices)
        option_price[:, -1] = payoff

        if exercise_type == 'european':
            exercise_binary = np.zeros(self.nfuture + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(self.nfuture + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(self.nfuture + 1)
            for i in range(self.nfuture+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Work backward through the tree
        for i in range(self.nfuture - 1, -1, -1):

            #expectation of discounted option prices in the future
            continuation = np.exp(-self.rfr * dt) * (self.p * payoff[1:] + (1 - self.p) * payoff[:-1])

            # Recalculate stock prices at this step
            S_maturity = Saverages[:i+1, i] # Going one step backward in the tree

            if exercise_binary[i] == 1:
                # Apply early exercise condition
                if option_type == 'call':
                    payoff = np.maximum(continuation, S_maturity - strike)
                    option_price [:len(payoff), i] = payoff
                else:
                    payoff = np.maximum(continuation, strike - S_maturity)
                    option_price [:len(payoff), i] = payoff
            else:
                payoff = continuation
                option_price [:len(continuation), i] = payoff

        # Final result at the root of the tree
        cal_price = payoff[0]
        return cal_price, option_price

    def compute_greeks(self, option_price, simulated_prices, strike, option_type, exercise_type, exercise_days):
        # Compute Greeks
        dt = 1 / 252
        S_tree =  simulated_prices
        option_tree = option_price

       # Delta using first step
        option_up = option_tree[1, 1]
        option_down = option_tree[0, 1]
        S_up = S_tree[1, 1]
        S_down = S_tree[0, 1]
        delta = (option_up - option_down) / (S_up - S_down)

        # Gamma using second step
        option_uu = option_tree[2, 2]
        option_ud = option_tree[1, 2]
        option_dd = option_tree[0, 2]
        S_uu = S_tree[2, 2]
        S_ud = S_tree[1, 2]
        S_dd = S_tree[0, 2]
        gamma_num = ((option_uu - option_ud) / (S_uu - S_ud) - (option_ud - option_dd) / (S_ud - S_dd))
        gamma_den = (0.5 * (S_uu - S_dd))
        gamma = gamma_num / gamma_den

        # Theta using option value decay
        theta = (option_tree[0, 2] - option_tree[0, 0]) / (2 * dt)

        # Vega and Rho approximations using shifts
        eps_sigma = 0.01
        eps_rfr = 0.0001

        # Save current state
        orig_sigma = np.log(self.u) / np.sqrt(dt)

        # Vega
        vega_class = exact_binomial_tree(self.df, self.nfuture, self.last_price ,self.npath, self.rfr, self.risk_neutral)

        sigma_up = orig_sigma + eps_sigma
        vega_simulated_price_up = vega_class.simulate_future_prices_CRR(self.rfr, sigma_up)
        vega_option_up, _ = vega_class.price_vanilla_options(vega_simulated_price_up, strike, option_type, exercise_type, exercise_days)

        sigma_down = orig_sigma - eps_sigma
        vega_simulated_price_down = vega_class.simulate_future_prices_CRR(self.rfr, sigma_down)
        vega_option_down, _ = vega_class.price_vanilla_options(vega_simulated_price_down, strike, option_type, exercise_type, exercise_days)

        vega = (vega_option_up - vega_option_down) / (2 * eps_sigma)

        # Rho
        rfr_up = self.rfr + eps_rfr
        rho_class_up = exact_binomial_tree(self.df, self.nfuture, self.last_price ,self.npath, rfr_up, self.risk_neutral)
        rho_simulated_price_up = rho_class_up.simulate_future_prices_CRR(rfr_up, orig_sigma)
        rho_option_up, _ = rho_class_up.price_vanilla_options(rho_simulated_price_up, strike, option_type, exercise_type, exercise_days)

        rfr_down = self.rfr - eps_rfr
        rho_class_down = exact_binomial_tree(self.df, self.nfuture, self.last_price ,self.npath, rfr_down, self.risk_neutral)
        rho_simulated_price_down = rho_class_down.simulate_future_prices_CRR(rfr_down, orig_sigma)
        rho_option_down, _ = rho_class_down.price_vanilla_options(rho_simulated_price_down, strike, option_type, exercise_type, exercise_days)

        rho = (rho_option_up - rho_option_down) / (2 * eps_rfr)

        return delta, gamma, theta, vega, rho
